In [8]:
import src.config as config
import pytz

In [9]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime("2024-03-01").floor("h").tz_localize(None)
print(f"{current_date=}")

# we fetch raw data for the last 28 days, to add redundancy to our pipeline
fetch_data_to = current_date  # Timezone-naive
fetch_data_from = fetch_data_to - timedelta(days=28)


current_date=Timestamp('2024-03-01 00:00:00')


In [12]:
from src.data import load_raw_data

def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e., 1 year)
    """
    from_date_ = from_date - timedelta(days=7*52)
    to_date_ = to_date - timedelta(days=7*52)
    print(f'{from_date=}, {to_date_=}')

    # download 2 files from website
    rides = load_raw_data(year=from_date_.year, months=from_date_.month)
    rides_2 = load_raw_data(year=to_date_.year, months=to_date_.month)

    # Ensure pickup_datetime fields are timezone-naive for comparison
    rides['pickup_datetime'] = pd.to_datetime(rides['pickup_datetime']).dt.tz_localize(None)
    rides_2['pickup_datetime'] = pd.to_datetime(rides_2['pickup_datetime']).dt.tz_localize(None)

    # Filter based on the from_date_ and to_date_
    rides = rides[rides.pickup_datetime >= from_date_]
    rides_2 = rides_2[rides_2.pickup_datetime < to_date_]

    rides = pd.concat([rides, rides_2])

    # shift the data to pretend this is recent data
    rides['pickup_datetime'] += timedelta(days=7*52)

    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [13]:
rides = fetch_batch_raw_data(from_date= fetch_data_from, to_date= fetch_data_to)

from_date=Timestamp('2024-02-02 00:00:00+0000', tz='UTC'), to_date_=Timestamp('2023-03-03 00:00:00+0000', tz='UTC')
File 2023-02 was already in local storage
File 2023-03 was already in local storage


TypeError: Invalid comparison between dtype=datetime64[us] and Timestamp

In [ ]:
from src.data import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 261/261 [00:00<00:00, 325.09it/s]


In [ ]:
# string to datetime
ts_data['pickup_hour'] = pd.to_datetime(ts_data['pickup_hour'], utc=True)
ts_data['pickup_location_id']=ts_data['pickup_location_id'].astype('Int64')
# add column with Unix epoch milliseconds
# ts_data['pickup_hour'] = ts_data['pickup_hour'].astype('int64').astype('int64') // 10**6


In [18]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value= config.HOPSWORKS_API_KEY
)

# connect to the feature store

feature_store = project.get_feature_store()


# connect to the feature group

feature_group = feature_store.get_or_create_feature_group(
                name= config.FEATRURE_GROUP_NAME,
                version = config.FEATRURE_GROUP_VERSION,
                description = 'Time-series data at hourly frequency',
                primary_key = ["pickup_location_id","pickup_hour"],
                event_time = "pickup_hour"
                )

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/696468
Connected. Call `.close()` to terminate connection gracefully.


In [19]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 0.00% |          | Rows 0/175392 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/696468/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x20d544bb730>, None)